In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt, argparse, numpy as np, math, sys, copy
from skimage.segmentation import slic, mark_boundaries, felzenszwalb
from skimage.util import img_as_float
from skimage import io
import numpy as np
import superpixels as sp
import glob
import pipeline as pipe
import time
import superpixels as sp
import scipy.stats as sci
from PIL import Image
from scipy import ndimage
import cv2

In [3]:
import argparse
import LoadBatches
from Models import Segnet, Unet, VGGSegnet, VGGUnet, FCN8, FCN32

Using TensorFlow backend.


### Parameters

In [11]:
#train path and batch size
train_images_path = '/media/falreis/falreis1/mestrado/deep_learning/BSDS500/pre_process/X/images/'
train_segs_path = '/media/falreis/falreis1/mestrado/deep_learning/BSDS500/pre_process/X/colorTruth/'
train_batch_size = 2 #args.batch_size

#val path and batch size
val_images_path = '/media/falreis/falreis1/mestrado/deep_learning/BSDS500/pre_process/images/val/'
val_segs_path = '/media/falreis/falreis1/mestrado/deep_learning/BSDS500/pre_process/colorTruth/val/'
val_batch_size = 2 #args.val_batch_size

#weights
save_weights_path = '/media/falreis/falreis1/mestrado/deep_learning/weights/'
load_weights = '' #args.load_weights

#input height and width
input_height = 481 #args.input_height
input_width = 481 #args.input_width

#classes, validade and epochs
n_classes = 2 #args.n_classes
validate = 'store_false' #args.validate
epochs = 10 # args.epochs

#optmizer and model name
optimizer_name = 'adadelta' #args.optimizer_name
model_name = 'segnet' #args.model_name

### Train

In [12]:
modelFns = { 'segnet':Segnet.Segnet
            , 'unet':Unet.Unet
            , 'vgg_segnet':VGGSegnet.VGGSegnet
            , 'vgg_unet':VGGUnet.VGGUnet
            , 'vgg_unet2':VGGUnet.VGGUnet2
            , 'fcn8':FCN8.FCN8
            , 'fcn32':FCN32.FCN32   
}
modelFN = modelFns[ model_name ]

m = modelFN(n_classes , input_height=input_height, input_width=input_width)
m.compile(loss='categorical_crossentropy', optimizer= optimizer_name, metrics=['accuracy'])

if len( load_weights ) > 0:
    m.load_weights(load_weights)

print("Model output shape" ,  m.output_shape)

output_height = m.outputHeight
output_width = m.outputWidth

G  = LoadBatches.imageSegmentationGenerator(train_images_path
                                            , train_segs_path
                                            , train_batch_size
                                            , n_classes
                                            , input_height
                                            , input_width
                                            , output_height
                                            , output_width
)

if validate:
    G2  = LoadBatches.imageSegmentationGenerator(val_images_path
                                                 , val_segs_path
                                                 , val_batch_size
                                                 , n_classes
                                                 , input_height
                                                 , input_width
                                                 , output_height
                                                 , output_width
)

if not validate:
    for ep in range(epochs):
        m.fit_generator(G, 64, epochs=1)
        m.save_weights(save_weights_path + "." + str(ep))
        m.save(save_weights_path + ".model." + str(ep))
else:
    for ep in range(epochs):
        m.fit_generator(G, 64, validation_data=G2, validation_steps=100, epochs=1)
        m.save_weights(save_weights_path + "." + str(ep))
        m.save(save_weights_path + ".model." + str(ep))

Model output shape (None, 3840, 2)
Epoch 1/1
64/64 [==============================] - 108s 2s/step - loss: 0.6150 - acc: 0.6744 - val_loss: 6.4979 - val_acc: 0.0095
Epoch 1/1
64/64 [==============================] - 105s 2s/step - loss: 0.0759 - acc: 0.9887 - val_loss: 6.4125 - val_acc: 0.0886
Epoch 1/1
64/64 [==============================] - 105s 2s/step - loss: 0.0198 - acc: 0.9951 - val_loss: 7.1700 - val_acc: 0.1017
Epoch 1/1
64/64 [==============================] - 105s 2s/step - loss: 0.0229 - acc: 0.9949 - val_loss: 7.2290 - val_acc: 0.1279
Epoch 1/1
64/64 [==============================] - 105s 2s/step - loss: 0.0014 - acc: 0.9972 - val_loss: 7.2324 - val_acc: 0.1636
Epoch 1/1
64/64 [==============================] - 105s 2s/step - loss: 3.7775e-04 - acc: 0.9971 - val_loss: 6.1412 - val_acc: 0.2713
Epoch 1/1
64/64 [==============================] - 107s 2s/step - loss: 1.7603e-04 - acc: 0.9971 - val_loss: 7.2339 - val_acc: 0.1426
Epoch 1/1
64/64 [==============================